In [37]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv('exponential.csv')
df.head()

,key,value
0,55,0
1,206,0
2,1542,0
3,1780,0
4,1809,0


In [52]:
X = df['key'].values.reshape(-1,1)
y = df['value'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [58]:
model = Sequential()
model.add(Dense(32, activation='relu',input_shape=(1,)))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_absolute_error')

In [59]:
model.fit(X_train,y_train,epochs=16,validation_split=0.1,batch_size=512)

Train on 81000 samples, validate on 9000 samples
Epoch 1/16
81000/81000 [==============================] - 0s 6us/step - loss: 499.3530 - val_loss: 493.0039
Epoch 2/16
81000/81000 [==============================] - 0s 4us/step - loss: 496.9610 - val_loss: 491.8344
Epoch 3/16
81000/81000 [==============================] - 0s 4us/step - loss: 495.9279 - val_loss: 489.8432
Epoch 4/16
81000/81000 [==============================] - 0s 4us/step - loss: 494.1277 - val_loss: 488.6016
Epoch 5/16
81000/81000 [==============================] - 0s 4us/step - loss: 495.2877 - val_loss: 490.2018
Epoch 6/16
81000/81000 [==============================] - 0s 4us/step - loss: 494.5745 - val_loss: 489.3624
Epoch 7/16
81000/81000 [==============================] - 0s 4us/step - loss: 494.0892 - val_loss: 489.5196
Epoch 8/16
81000/81000 [==============================] - 0s 4us/step - loss: 493.8119 - val_loss: 488.3093
Epoch 9/16
81000/81000 [==============================] - 0s 4us/step - loss: 492.6240 

In [60]:
%%timeit
preds = model.predict(X_test)

138 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [67]:
print(model.predict(X_test[9999]))

[[6.9225965]]


In [61]:
preds.shape

(1000, 1)

In [57]:
errs = preds - y_test
max_err = 0
for i in errs:
    if i>max_err or i>(-max_err):
        max_err = i
print(max_err)

ValueError: operands could not be broadcast together with shapes (1000,1) (10000,1) 

In [35]:
class Item():
  def __init__(self, key, value):
    self.k = key
    self.v = value

  def __str__(self):
    return "Key: " + str(self.k) + " Value: " + str(self.v)

  def __eq__(self, other):
    return self.k == other.k

  def __gt__(self, other):
    return self.k > other.k

  def __ge__(self, other):
    return self.k >= other.k

  def __lt__(self, other):
    return self.k < other.k

  def __le__(self, other):
    return self.k <= other.k

# Every node in the B-Tree is implemented as a BTreeNode

class BTreeNode:

  """
  numberOfKeys - number of non-null keys in the node
  items - keys in a node
  children - indexes of nodes that are the children of the current node
  index - position of the node in sorted order
  """

  def __init__(self, keys_per_node = 0):
    self.items = [None] * keys_per_node
    self.children = [None] * (keys_per_node + 1)
    self.isLeaf = True
    self.numberOfKeys = 0
    self.index = None

  def set_index(self, index):
    self.index = index

  def search(self, b_tree, an_item):
    i = 0
    while i < self.numberOfKeys and an_item > self.items[i]:
      i += 1
    if i < self.numberOfKeys and an_item == self.items[i]:
      return {'found': True, 'fileIndex': self.index, 'nodeIndex': i}
    if self.isLeaf:
      return {'found': False, 'fileIndex': self.index, 'nodeIndex': i - 1}
    else:
      return b_tree.get_node(self.children[i]).search(b_tree, an_item)


# B-Tree stores the properties of the required B-Tree

class BTree:

  """
  keys_per_node - page_size (number of keys in each node)
  degree - Number of keys to half-fill the node
  rootNode - root node in the B-Tree
  nodes - map of all nodes in the B-Tree (index to node mapping)
  rootIndex - index of the rootNode in the nodes map
  freeIndex - An incremental counter always pointing to next position to be filled
  """

  def __init__(self, keys_per_node = 0):
    self.keys_per_node = keys_per_node
    self.degree = (self.keys_per_node + 1) // 2 #Should be int
    self.rootIndex = 1
    self.freeIndex = self.rootIndex + 1
    self.rootNode = BTreeNode(self.keys_per_node)
    self.rootNode.set_index(self.rootIndex)
    self.nodes = {}
    self.write_at(self.rootIndex, self.rootNode)
    

  def build(self, keys, values):
    if len(keys) != len(values):
      return
    for ind in range(len(keys)):
      self.insert(Item(keys[ind], values[ind]))

  def write_at(self, index, a_node):
    self.nodes[index] = a_node

  def representation(self):
    return "BTree("+str(self.degree)+",\n function" + str(self.function()) + ","+ str(self.rootIndex)+","+str(self.freeIndex)
  
  def get_free_index(self):
    self.freeIndex += 1
    return self.freeIndex - 1

  def set_root_node(self, r):
    self.rootNode = r
    self.rootIndex = self.rootNode.index

  def get_free_node(self):
    new_node = BTreeNode(self.keys_per_node)
    index = self.get_free_index()
    new_node.set_index(index)
    self.write_at(index, new_node)
    return new_node

  def print_nodes(self):
    s = ''
    for x in self.nodes:
      s = s + str(x) + ' '
    return s
    
  def function(self):
    return 'BTree Degree:' + str(self.degree) + ' RootIndex:' + str(self.rootIndex)+ ' FreeIndex:' +str(self.freeIndex) + '\nNodes:' + str(self.print_nodes()) 

  def predict(self, key):
    search_result = self.search(Item(key, 0))
    if not search_result['found']:
      return -1
    node = search_result['fileIndex']
    item = search_result['nodeIndex']
    return self.nodes[node].items[item].v

  def get_node(self, index):
    return self.nodes[index]

  def split_child(self, p_node, i, c_node):
    new_node = self.get_free_node()
    new_node.isLeaf = c_node.isLeaf
    new_node.numberOfKeys = self.degree - 1
    for j in range(0, self.degree - 1):
      new_node.items[j] = c_node.items[j + self.degree]
    if not c_node.isLeaf:
      for j in range(0, self.degree):
        new_node.children[j] = c_node.children[j + self.degree]
    c_node.numberOfKeys = self.degree - 1
    j = p_node.numberOfKeys + 1
    while j > i + 1:
      p_node.children[j + 1] = p_node.children[j]
      j -= 1
    p_node.children[j] = new_node.index
    j = p_node.numberOfKeys
    while j > i:
      p_node.items[j + 1] = p_node.items[j]
      j -= 1
    p_node.items[i] = c_node.items[self.degree - 1]
    p_node.numberOfKeys += 1

  def search(self, an_item):
     return self.rootNode.search(self, an_item)

  def insert(self, an_item):
    search_result = self.search(an_item)
    if search_result['found']:
      return None
    r = self.rootNode
    if r.numberOfKeys == 2 * self.degree - 1:
      s = self.get_free_node()
      self.set_root_node(s)
      s.isLeaf = False
      s.numberOfKeys = 0
      s.children[0] = r.index
      self.split_child(s, 0, r)
      self.insert_not_full(s, an_item)
    else:
      self.insert_not_full(r, an_item)

  def insert_not_full(self, inNode, anItem):
    i = inNode.numberOfKeys - 1
    if inNode.isLeaf:
      while i >= 0 and anItem < inNode.items[i]:
        inNode.items[i + 1] = inNode.items[i]
        i -= 1
      inNode.items[i + 1] = anItem
      inNode.numberOfKeys += 1
    else:
      while i >= 0 and anItem < inNode.items[i]:
        i -= 1
      i += 1
      if self.get_node(inNode.children[i]).numberOfKeys == 2 * self.degree - 1:
        self.split_child(inNode, i, self.get_node(inNode.children[i]))
        if anItem > inNode.items[i]:
          i += 1
      self.insert_not_full(self.get_node(inNode.children[i]), anItem)

In [36]:
page_size=64
btree = BTree(page_size)

AttributeError: 'BTreeNode' object has no attribute 'set_index'